[👈 Chapter 22](22-pandas-temporal-data.ipynb) -
[🏠 To index](README.md) -
[👉 Chapter 24](24-functions.ipynb)

# 23 - Accessing the Reddit API using the `praw` library

In [8]:
# In chapter 13 you've learned to work with the Wikipedia REST API. Many other sites also 
# have an API, such as Reddit. This API uses authentication, which means it's a bit harder than the
# Wikipedia API to use. Because of that, it's easier to use a library to do the difficult stuff for you. 
# We're going to use the 'praw' library. To install it, open a terminal and type 'pip install praw'.
# If you're working on Windows with the Anaconda package, try the 'Anaconda prompt' to do this.
# 
# You first need to create credentials for this app, and you also need to make a Reddit account first. 
# After that, follow this tutorial (http://www.storybench.org/how-to-scrape-reddit-with-python/)
# until the heading about the 'shebang line'.
#
# To use praw, we first need to import the library
import praw

# Also, import pandas for later in this chapter
import pandas as pd

In [9]:
# We need to define two 'keys' to use with the Reddit API. These are supposed to be secret, 
# so enter them here, don't share them with anyone! We also need a 'user agent string', 
# simply replace the place where it says 'YOUR_NAME' with your Reddit username
#
# Note how we CAPITALIZE the variable names. This is a convention
# to indicate that these are 'constants' and shouldn't be changed
CLIENT_ID = "CLIENT_ID_HERE"
CLIENT_SECRET = "CLIENT_SECRET_HERE"
USER_AGENT = f"python:{CLIENT_ID}:0.1 (by /u/YOUR_NAME)"

In [10]:
# Okay! Now we can create an instance of the api by using this command
api = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    user_agent = USER_AGENT
)

In [11]:
# To test the api, see if this returns 'True'
api.read_only

True

In [12]:
# Let's try getting the 3 'hottest' submission on the popular 'Ask Reddit' subreddit.
# 'Hot' means they're the most popular at the moment.
# Note the 'limit' argument, we can pass these with an equals sign, just like variable declaration
submissions = api.subreddit('askReddit').hot(limit = 3)
submissions

In [13]:
# As you can see this is an 'iterator', we can use 'for' to loop through these submissions
for sub in submissions:
    # Note how these objects don't use dictionaries, and you access
    # data by the dot notation. These data points are called object attributes.    
    print(sub.title)
    
    # You can even get a nested property this way
    print(sub.author.name)

We've heard all about red flags but, reddit, what are some green flags on a first date?
YaBoiLeeDawg
What is your "we will never speak of this again" moment?
dannyjayes1
What's the funniest way to answer a wrong number call?
PanicAtTheMetro


In [14]:
# Note that when you want to use the data from the Praw library in Pandas you need to convert it first to 
# a list with dictionaries
results = api.subreddit('askReddit').hot(limit = 10)
submissions = []
for result in results:
    submissions.append({
        "title" : result.title,
        "score" : result.score,
        "comments" : result.num_comments
    })

In [15]:
df = pd.DataFrame(submissions)
df.head()

,comments,score,title
0,5229,25815,"We've heard all about red flags but, reddit, w..."
1,1901,3985,"What is your ""we will never speak of this agai..."
2,507,673,What's the funniest way to answer a wrong numb...
3,2024,1419,What book gripped you to the point that it too...
4,22262,33601,What is the worst casting decision in the hist...


In [16]:
# There are lots of properties ('attributes') that come with every submission, you can use python's vars() 
# method to check them out. We need to convert the submissions to a list first to do that. We also need to
# re-do the method, because a 'generator' is over after one go
submissions = api.subreddit('askReddit').hot(limit = 3)
submissions = list(submissions)
vars(submissions[0])

{'_reddit': <praw.reddit.Reddit at 0x11504a0f0>,
 'approved_at_utc': None,
 'subreddit': Subreddit(display_name='AskReddit'),
 'selftext': '',
 'author_fullname': 't2_10j2qpmr',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': "We've heard all about red flags but, reddit, what are some green flags on a first date?",
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/AskReddit',
 'hidden': False,
 'pwls': 6,
 'link_flair_css_class': None,
 'downs': 0,
 'parent_whitelist_status': 'all_ads',
 'hide_score': False,
 'name': 't3_9wz259',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'author_flair_background_color': None,
 'subreddit_type': 'public',
 'ups': 25857,
 'domain': 'self.AskReddit',
 'media_embed': {},
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': False,
 'is_meta': False,
 'category': None,
 'secure_media_embed': {},
 'link_flair_text'

In [17]:
# Another way is to use the .json 'endpoint' that is available for most Reddit URLS, for example
# this page:
# < https://www.reddit.com/r/askreddit >
# Is also available in the JSON format like this:
# < https://www.reddit.com/r/askreddit.json?limit=10 >

In [18]:
# To get the comments for a submission you can use the 'comments' attribute of a submission,
# we do need to a little magic to filter out the comments that aren't really comments
# but 'read more' buttons
# We're just going to take the first submission from the list as an example
for comment in submissions[0].comments:
    # Check if this is a 'more comments' node, skip those
    # 'continue' wil go through the next loop
    if isinstance(comment, praw.models.MoreComments):
        continue
        
    # Limit the body text to the first 100 chars
    print(comment.body[0:100])

If either person does something ridiculously embarrassing and the other person laughs.  I guess ther
When you get to the point that you run out of ideas on stuff to do but you both want tokeep the date
Talking for 5 hours but only thinking the date lasted around 90 minutes
Cheeks sore from laughing
A friend of mine hit it off in the car with her date that they ended up driving over two hours PAST 
On our first date in high school, my wife tried to pay for the movie tickets and then the snacks at 
When they actually pay attention to what you're talking  about and even make the effort to come back
When you don't want it to end. 
My first date with my current girlfriend started as a casual meet up
For me, it was the retry worthy fuck ups. My first date with my now wife we had dinner and the host 
Genuine laughter.  I have anxiety and a lot of involuntary quirks, sometimes my defense mechanism fo
She seems genuinely curious about you. Asks about your life and your past etc. Instead of just

[👈 Chapter 22](22-pandas-temporal-data.ipynb) -
[🏠 To index](README.md) -
[👉 Chapter 24](24-functions.ipynb)